In [64]:
!pip install --upgrade pip
!pip uninstall transformers -y #uninstall version you have currently, just bc we might need diff versions for diff fine tuning models
#!pip install transformers==4.18.0 #note that we need an older version of transformers to use this model
!pip install transformers 
!pip install --no-cache-dir transformers sentencepiece
!pip install datasets
!pip install accelerate -U




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [65]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset, load_metric
import torch
import pandas as pd
import numpy as np
#from google.colab import files
#from google.colab import drive
import glob
import zipfile
from sklearn.model_selection import train_test_split




In [66]:
def compute_metrics(p):
    metric = load_metric("accuracy")
    logits, labels = p
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [67]:
# Instantiate the tokenizer for the french model
tokenizer = AutoTokenizer.from_pretrained("cmarkea/distilcamembert-base-sentiment")


def tokenize_function(examples, tokenizer=tokenizer):
    """
    Function to tokenize the data.
    examples : data to tokenize ; dict
    tokenizer : tokenizer to use ; DistilBertTokenizer
    """
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

In [68]:
def model(train_comments, train_labels,
                    val_comments, val_labels, test_comments,
                    test_labels,
                    batch_size_train, batch_size_val,
                    epochs, tokenizer=tokenizer, learning_rate=1e-4, weight_decay=0.001):
    """
    Function to train a  model on the data.
    train_comments : comments for training ; lst of str
    train_labels : labels for training ; lst of int
    val_comments : comments for validation ; lst of str
    val_labels : labels for validation ; lst of int
    batch_size_train : batch size for training ; int
    batch_size_val : batch size for validation ; int
    epochs : number of epochs ; int
    tokenizer : tokenizer to use
    """
    # Set the device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = AutoModelForSequenceClassification.from_pretrained("cmarkea/distilcamembert-base-sentiment")
    model.to(device) # French model has no gpu acceleration

    # Setup the Hugging Face Dataset Class
    train_dataset_dict = {"text": train_comments, "label": train_labels}
    val_dataset_dict = {"text": val_comments, "label": val_labels}
    test_dataset_dict = {"text": test_comments, "label": test_labels}

    train_dataset = Dataset.from_dict(train_dataset_dict)
    val_dataset = Dataset.from_dict(val_dataset_dict)
    test_dataset = Dataset.from_dict(test_dataset_dict)

    # Apply the tokenizer to the datasets
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Remove columns we do not need for training
    train_dataset = train_dataset.remove_columns(["text"])
    val_dataset = val_dataset.remove_columns(["text"])
    test_dataset = test_dataset.remove_columns(["text"])

    # Set the format of the datasets to PyTorch tensors
    train_dataset.set_format("torch")
    val_dataset.set_format("torch")
    test_dataset.set_format("torch")


    # Training arguments
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=epochs,              # total number of training epochs
        per_device_train_batch_size=batch_size_train,  # batch size for training
        per_device_eval_batch_size=batch_size_val,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=weight_decay,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
        evaluation_strategy="steps",     # Evaluate every `eval_steps`
        eval_steps=10,                   # Number of steps between evaluations
        save_steps=10,                   # Save the model every `save_steps`
        load_best_model_at_end=True,     # Load the best model at the end of training#
        learning_rate= learning_rate,              # Set the learning rate
        metric_for_best_model="eval_loss", # Use evaluation loss to check how good our model is performing
        greater_is_better=False,
    )

    # Trainer
    trainer = Trainer(
        model=model,                         # model
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=val_dataset,            # evaluation dataset
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=10)], # Early Stopping for Overfitting
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()

    # Evaluate the model on the test set
    test_results = trainer.evaluate(eval_dataset=test_dataset)

    # Get test accuracy
    print("Test Accuracy:", test_results['eval_accuracy'])



    return model, tokenizer, test_results



In [46]:
def save_model(model, tokenizer, path):
    """
    Function to save the model
    model : model to save ;
    tokenizer : tokenizer to save ;
    path : path to save the model ; str
    """

    model_save_path =  path
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

In [ ]:
# Open a file upload dialog
# Select here all files to upload!
# If already uploaded, just press 'Cancel Upload'
# They are contained within a zip file.
uploaded = files.upload()

In [ ]:
# Set the path to the data
# On local machine use the relative path, for example
# path = 'NLP labelled data preview/english set/'
# On Google Colab use this path
# '/content/'
path = '/content/'

In [ ]:
# Unzip the folder
with zipfile.ZipFile(path + 'french_data_finetuning.zip', 'r') as zip_ref:
    zip_ref.extractall('french_data_finetuning')

In [ ]:
# Load the dataset
all_comments_with_labels = glob.glob(path + 'french_data_finetuning/french_data_finetuning/*.csv')

In [47]:
# TODO : remove, just testing purposes on local machine
all_comments_with_labels = pd.read_csv('/Users/marlon/VS-Code-Projects/Youtube/Comments DB/French/finetuning/1_INFERENCE_FRENCH_norep_cleaned_and_filtered_comments_for_INFERENCE.csv')

In [48]:
# Read in the data
all_comments_with_labels = pd.concat([pd.read_csv(f) for f in all_comments_with_labels], ignore_index = True)

FileNotFoundError: [Errno 2] No such file or directory: 'Unnamed: 0'

In [49]:
import random
# FOR TESTING PURPOSES (since we don't have the labels for french yet)
# Add a new column to the dataframe all_comments_with_labels and fill it up with integer values randomly distributed between 0 and 2
all_comments_with_labels['label'] = [random.randint(0, 1) for i in range(len(all_comments_with_labels))]

In [50]:
# Conversion
# Since this model is trained with labels 1 to 5 stars, we will adjust our labels :
# We will refer to negative as 2 stars and positive as 4 stars
# TODO : conversion by confidence level

all_comments_with_labels['label'] = all_comments_with_labels['label'].replace({0: 2, 1: 4})


In [51]:
# Seperate the two columns in the dataframe into comments and labels and turn them into lists
comments = all_comments_with_labels['Comment'].tolist()
labels = all_comments_with_labels['label'].tolist()


In [54]:
# Convert the labels to integers
labels = [int(label) for label in labels]
# Turn all elements in comments into strings
comments = [str(comment) for comment in comments]


In [55]:
# Assess how many comments we have
print("Number of comments: ", len(comments))
# Asses how many negative (0) , neutral (1) and positive (2) comments we have
print("Number of negative comments: ", labels.count(2))
print("Number of neutral comments: ", labels.count(4))
#print("Number of positive comments: ", labels.count(4))


Number of comments:  2159
Number of negative comments:  1051
Number of neutral comments:  1108


In [56]:
# Split the data into a train and test set with stratification
train_comments, temp_comments, train_labels, temp_labels = train_test_split(
    comments, labels, test_size=0.3, random_state=42, stratify=labels)



In [57]:
# Split the data into training and validation sets with stratification
val_comments, test_comments, val_labels, test_labels = train_test_split(temp_comments, temp_labels, test_size=0.1, random_state=42, stratify= temp_labels)

In [58]:
# Look at the number of comments in the training and validation sets
print("Number of comments in the training set: ", len(train_comments))
print("Number of comments in the validation set: ", len(val_comments))
print("Number of comments in the test set: ", len(test_comments))
# Look at labels in the training and validation sets
print("Number of negative comments in the training set: ", train_labels.count(2))
print("Number of neutral comments in the training set: ", train_labels.count(3))
print("Number of positive comments in the training set: ", train_labels.count(4))
print("Number of negative comments in the validation set: ", val_labels.count(2))
print("Number of neutral comments in the validation set: ", val_labels.count(3))
print("Number of positive comments in the validation set: ", val_labels.count(4))
print("Number of negative comments in the test set: ", test_labels.count(2))
print("Number of neutral comments in the test set: ", test_labels.count(3))
print("Number of positive comments in the test set: ", test_labels.count(4))


Number of comments in the training set:  1511
Number of comments in the validation set:  583
Number of comments in the test set:  65
Number of negative comments in the training set:  736
Number of neutral comments in the training set:  0
Number of positive comments in the training set:  775
Number of negative comments in the validation set:  283
Number of neutral comments in the validation set:  0
Number of positive comments in the validation set:  300
Number of negative comments in the test set:  32
Number of neutral comments in the test set:  0
Number of positive comments in the test set:  33


In [69]:
# Train the model
model_trained, tokenizer_trained, test_results = model(train_comments, train_labels, val_comments, val_labels,test_comments, test_labels, batch_size_train = 128, batch_size_val = 32, epochs = 1, tokenizer = tokenizer, learning_rate=1e-4, weight_decay=0.001)


Map:   0%|          | 0/1511 [00:00<?, ? examples/s]

Map:   0%|          | 0/583 [00:00<?, ? examples/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

/opt/homebrew/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/12 [00:00<?, ?it/s]

...

/opt/homebrew/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 1.952553391456604, 'eval_accuracy': 0.2727272727272727, 'eval_runtime': 2.0099, 'eval_samples_per_second': 290.057, 'eval_steps_per_second': 9.453, 'epoch': 0.83}
{'train_runtime': 13.7404, 'train_samples_per_second': 109.968, 'train_steps_per_second': 0.873, 'train_loss': 2.063450654347738, 'epoch': 1.0}


  0%|          | 0/3 [00:00<?, ?it/s]

Test Accuracy: 0.2153846153846154


/opt/homebrew/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# Save the model
save_model(model_trained, tokenizer_trained, path + 'french_model')


In [ ]:
# Save the fine-tuned model to your system

# Mount to drive
drive.mount('/content/drive')

!cp -r /content/french_model /content/drive/MyDrive/

# Now download it from your Google Drive Account !